# Triage Amadey Loader
> Triage of Amadey loader

- toc: true 
- badges: true
- categories: [triage,malware,amadey,loader,config]

## Overview

Unknown sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/7ff0ff6e51a58398ad73da3cc8e7e6233a23e49d93aaa4b190672e4f9f08b9bb//). Some of the sandbox tags labeled it as **amadey**, we are going to validate this.

### Samples
- [Packed Sample](https://malshare.com/sample.php?action=detail&hash=7ff0ff6e51a58398ad73da3cc8e7e6233a23e49d93aaa4b190672e4f9f08b9bb): `7ff0ff6e51a58398ad73da3cc8e7e6233a23e49d93aaa4b190672e4f9f08b9bb`
- [Unpacked Sample](https://malshare.com/sample.php?action=detail&hash=6e01f9d1997186d06274a508bc0a511aa6fb50e430b77efca593c00d3fc62cba): `6e01f9d1997186d06274a508bc0a511aa6fb50e430b77efca593c00d3fc62cba`

## References
- ANY.RUN detects as [amadey](https://app.any.run/tasks/667fda8d-a094-4e42-8caa-e9dad26186ad)
- Intezer detects as [deyma](https://analyze.intezer.com/analyses/9a751084-db27-441a-b86c-a31d49eac09c)
- JoeSandbox detects as [amadey](https://www.joesandbox.com/analysis/1003304)
- Hatching detects as [amadey](https://tria.ge/reports/220529-v39wtscgdp/)
- [Analyzing Amadey Loader](https://nao-sec.org/2019/04/Analyzing-amadey.html)

According to [Malpedia](https://malpedia.caad.fkie.fraunhofer.de/details/win.amadey)
> Amadey is a botnet that appeared around October 2018 and is being sold for about 500$ on Russian-speaking hacking forums. It periodically sends information about the system and installed AV software to its C2 server and polls to receive orders from it. Its main functionality is that it can load other payloads (called "tasks") for all or specifically targeted computers compromised by the malware.


## Analysis

### Artifacts 
- PDB string `D:\Mktmp\NL1\Release\NL1.pdb`
- Compiled with `Microsoft Visual C/C++`

In [2]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

    
def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

In [34]:
key_buff = b'6768875d0dd576a718d85aa1d71d25c1\x00'
data_buff = b'6679AA9D98A9C95093B7785783AA85868CA95864AEB051955760B88055'
out = []

def decrypt_string(data, key):
    data = unhex(data)
    key = key[1:]
    out = []
    for i in range(len(data)):
        out.append((data[i] - key[i % len(key)]) & 0xff)
    return bytes(out)

print(decrypt_string(data_buff, key_buff))

b'/Create /SC MINUTE /MO 1 /TN '


In [43]:
import re


FILE_PATH = '/tmp/amadey_unpacked.bin'
data = open(FILE_PATH, 'rb').read()


def get_strings():
    string_egg = rb'(([A-F0-9]{2}){4,})\x00'
    enc_strings = []
    for m in re.finditer(string_egg, data):
        enc_str = m.group(1)
        enc_strings.append(enc_str)
    return enc_strings
    
def get_keys():
    key_egg = rb'([a-f0-9]{2}){8,}\x00'
    keys = []
    for m in re.finditer(key_egg, data):
        key_str = m.group(0)
        keys.append(key_str)
    return keys



In [46]:

is_ascii = lambda s: re.match(b'^[\s!-~]+$', s)


def validate_key(keys, strings):
    canary_str = max(strings, key=len)
    for key in keys:
        if is_ascii(decrypt_string(canary_str, key)):
            return key
    return None


def decrypt_strings(data):
    out = []
    enc_strings = get_strings()
    keys = get_keys()
    key = validate_key(keys, enc_strings)
    if key is not None:
        for s in enc_strings:
            out.append(decrypt_string(s, key))
    return out

strings = decrypt_strings(data)
for s in strings:
    print(s)
        


b'happyday9risce.com'
b'/gg4mn3s/index.php'
b'xksldjf9sksdjfks.com'
b'dhisa8f9ah02hopasiaf.com'
b'3.20'
b'a10b8dfb5f'
b'orxds.exe'
b'SCHTASKS'
b'/Create /SC MINUTE /MO 1 /TN '
b' /TR "'
b'" /F'
b'/C REG ADD "HKCU\\Software\\Microsoft\\Windows\\CurrentVersion\\Explorer\\User Shell Folders" /f /v Startup /t REG_SZ /d '
b'SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Explorer\\Shell Folders'
b'Startup'
b'/C REG ADD "HKCU\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\RunOnce" /f /v rem /t REG_SZ /d '
b'"cmd /C RMDIR /s/q "'
b'SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run'
b'rundll32 '
b' /Delete /TN "'
b'powershell'
b"-Command $action = New-ScheduledTaskAction -Execute '"
b"'; $time = New-TimeSpan -Minutes 1; $trigger = New-ScheduledTaskTrigger -Once -At 0:00 -RepetitionInterval $time; Register-ScheduledTask -Action $action -Trigger $trigger -TaskName '"
b"-Command $action = Unregister-ScheduledTask -TaskName '"
b'-Confirm: $false'
b'POST'
b'&vs='
b'&sd='
b'&os='
b'&bi='
b'&ar='
b'&pc